# Induced gap and magnetic field

In [ ]:
# 1. Standard library imports
import os.path
from functools import partial
from operator import itemgetter

# 2. External package imports
import holoviews as hv
import kwant
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from toolz import partition_all

# 3. Internal imports
import funcs

hv.notebook_extension('matplotlib')
print(kwant.__version__)
import warnings
warnings.filterwarnings('ignore', r'Interpreting')

# Connect to ipyparallel

In [ ]:
from ipyparallel import Client
import hpc05
client = hpc05.Client(profile='pbs', timeout=60, extra_args='--timeout=1200')
# client = Client()
dview = client[:]
dview.use_dill()
lview = client.load_balanced_view()
print(len(dview))

%px import sys, os; sys.path.append(os.path.expanduser('~/Work/induced_gap_B_field/'))
%px import funcs

# Usage

In [ ]:
params = dict(alpha=20, B_x=0.5, B_y=0, B_z=0, Delta=60, g=50, orbital=True,
              mu=7, mu_lead=7, c_tunnel=3/4, V=lambda x, y, z: 0, V_barrier=0, **funcs.constants.__dict__)

syst_pars = dict(a=10, angle=0, onsite_disorder=False,
                 L=200, coverage_angle=180, r1=35, r2=70, shape='circle',
                 with_leads=True, with_shell=True, A_correction=True)

syst = funcs.make_3d_wire(**syst_pars)

kwant.plot(syst);

In [ ]:
lead = syst.leads[0]
funcs.plot_wfs_in_cross_section(lead, params, 0)

In [ ]:
%%opts Image [colorbar=True]

x = 20
xy = funcs.get_cross_section(syst, x, 0)

V = lambda x, y, z: 10 * z / syst_pars['r1']

potential = np.array([V(*s.pos) for s in syst.sites if s.pos[0]==x])

hv.Image(np.rot90(kwant.plotter.mask_interpolate(xy, potential, oversampling=1)[0]))

In [ ]:
%%opts Path [aspect='square']
params = dict(c_tunnel=3/4, B_x=0, B_y=0, B_z=0, V_barrier=50, g=50,
              alpha=20, orbital=True, V=lambda x, y, z: 0, Delta=60,
              **funcs.constants.__dict__)

lead_pars = dict(a=10, angle=0,
                 coverage_angle=180, r1=35, r2=70, shape='circle',
                 with_shell=True, A_correction=True)

lead = funcs.make_lead(**lead_pars).finalized()
params['mu_lead'] = 10
params['B_y'] = 0
ks = np.linspace(-1, 1)
Es = funcs.bands(lead, params, ks)
p1 = hv.Path((ks, Es))[:, -100:100]

p1[:, -25:25] + p1[:, -2:2]

In [ ]:
%%time
gap = funcs.find_gap(lead, params)
print('The bandgap is {} meV'.format(gap))

In [ ]:
params = dict(alpha=20, B_x=0, B_y=0, B_z=0, Delta=60, g=50, orbital=True,
              mu=15, mu_lead=15, c_tunnel=3/4, V=lambda x,y,z:0, V_barrier=50, **funcs.constants.__dict__)

S = kwant.smatrix(syst, params=params)

In [ ]:
funcs.andreev_conductance(syst, params)

# Tuning the gap

In [ ]:
params = dict(c_tunnel=3/4, B_x=0, B_y=0, B_z=0, V_barrier=50, g=50,
              alpha=20, orbital=True, V='lambda x, y, z: 0',
              **funcs.constants.__dict__)

syst_pars = dict(a=10, angle=0, coverage_angle=180, r1=35, r2=70,
                 shape='circle', with_shell=True, A_correction=True)

def func(x, syst_pars=syst_pars, params=params):
    import funcs
    import numpy as np
    lead = funcs.make_lead(**syst_pars).finalized()
    params['mu_lead'], params['Delta'] = x
    # Combine the fixed parameters `params` and the changing
    # parameters `val` to one dict and evaluate the string
    # lambda functions.
    params = funcs.parse_params(params)

    # Create the Hamiltonian `ham` at k=0.
    h0 = lead.cell_hamiltonian(params=params)
    t0 = lead.inter_cell_hopping(params=params)
    ham = h0 + t0 + t0.conj().T

    # Find the energies.
    ev = np.linalg.eigvalsh(ham)
    
    # Return a combined dictionary with the results and input.
    return np.abs(ev).min()

learner = adaptive.Learner2D(func, [(5, 15), (0, 100)], loss)

In [ ]:
runner = adaptive.Runner(learner, client, goal=lambda l: l.loss() < 0.001)

In [ ]:
plot = learner.plot().redim(x='mu', y='Delta', z='E_gap')
plot

In [ ]:
plot.reduce(['mu'], np.min)

In [ ]:
Deltas = np.arange(*learner.bounds[1])
hm = hv.HoloMap({Delta: plot.sample(Delta=Delta)[:, 0.3:0.7] for Delta in Deltas})
hm * hv.HLine(0.4)

# adaptive

In [ ]:
import adaptive
from adaptive.learner.learner2D import deviations, areas
adaptive.notebook_extension()

def loss(ip):
    A = np.sqrt(areas(ip))
    dev = deviations(ip)[0]
    return A * dev + A**2

In [ ]:
params = dict(Delta=60, c_tunnel=3/4, V_barrier=50,
              V='lambda x, y, z: 0',
             **funcs.constants.__dict__)

syst_pars = dict(a=10, angle=0, onsite_disorder=False,
                 L=2000, coverage_angle=180, r1=35, r2=70, shape='circle',
                 with_leads=True, with_shell=True, A_correction=True)

vals = funcs.named_product(g=[0, 50],
                           alpha=[0, 20],
                           orbital=[False, True],
                           direction=list('xyz'),
                           mu=list(range(10, 21)))

def func(x, val, syst_pars=syst_pars, params=params):
    import funcs
    params = funcs.parse_params(dict(**params, **val))
    
    params['mu_lead'] = params['mu']
    params['B_x'], params['B_y'], params['B_z'] = (0, 0, 0)
    val['V_bias'], params['B_{}'.format(val['direction'])] = x

    syst = funcs.make_3d_wire(**syst_pars)
    return dict(**funcs.andreev_conductance(syst, params, E=val['V_bias']), **val)

learners = [adaptive.Learner2D(partial(func, val=val), [(-1, 1), (0, 1)], loss)
            for val in vals]

learner = adaptive.DataSaver(adaptive.BalancingLearner(learners), itemgetter('G_01'))

In [ ]:
import deepdish as dd
data = dd.io.load('data.dd')
for l, (val, d) in zip(learners, data):
    l.add_data(d['points'], d['values'])

In [ ]:
runner = adaptive.Runner(learner, executor=client, 
                         shutdown_executor=False)

In [ ]:
learner.learner.learners[111].plot(triangles_alpha=0.2)

In [ ]:
%%opts Image {+framewise} [colorbar=True]
%%output size=200

plots = {tuple(val.values()): l.plot(100, 100, 0.5) for (l, val) in zip(learner.learner.learners, vals)}
hm = hv.HoloMap(plots, kdims=list(vals[0].keys()))
hm

In [ ]:
import deepdish as dd
data = [(val, {'points': learner.points, 'values': learner.values})
        for learner, val in zip(learner.learner.learners, vals)]
dd.io.save('data.dd', data)

# ipyparallel

In [ ]:
# Make sure this folder is in your $PYTHONPATH
try:
    # If this can be imported, it assumes you are on the TU Delft network with access to cluster
    import hpc05
    client = hpc05.Client(profile='pbs', timeout=60, extra_args='--timeout=1200')
    print("Connected to hpc05")
except ImportError:
    from ipyparallel import Client
    client = Client()
    print("Connected to local engines")

In [ ]:
dview = client[:]
dview.use_dill()
lview = client.load_balanced_view()
print(len(dview))

%px import sys, os; sys.path.append(os.path.expanduser('~/Work/induced_gap_B_field/'))
%px import funcs

![](https://upload.wikimedia.org/wikipedia/commons/thumb/4/4f/3D_Spherical.svg/200px-3D_Spherical.svg.png)

In [ ]:
params = dict(Delta=60, c_tunnel=3/4, V_barrier=50, V='lambda x, y, z: 0', **funcs.constants.__dict__)

syst_pars = dict(a=10, onsite_disorder=False,
                 L=2000, coverage_angle=180, r1=35, r2=70, shape='circle',
                 with_leads=True, with_shell=True, A_correction=True)


vals = funcs.named_product(B=np.linspace(0, 2, 51),
                           direction=list('xyz'),
                           V_bias=np.linspace(-0.25, 0.25, 51),
                           g=[0, 50],
                           alpha=[0, 20],
                           orbital=[False, True],
                           mu=range(10, 21),
                           angle=[0, 45, 90])

vals = [val for val in vals if (not (val['theta'] == 0 and val['phi'] != 0)) and
                               (not (val['g'] == 0 and val['orbital'] == False))]


def func(val, syst_pars=syst_pars, params=params):
    import funcs
    params = funcs.parse_params(dict(**params, **val))
    
    params['mu_lead'] = params['mu']
    params['B_x'], params['B_y'], params['B_z'] = (0, 0, 0)
    params['B_{}'.format(val['direction'])] = val['B']
    
    # Remove parameters from `params` that belong in `syst_pars`.
    for x in ['angle']:
        syst_pars[x] = params.pop(x)

    syst = funcs.make_3d_wire(**syst_pars)
    return dict(**funcs.andreev_conductance(syst, params, E=val['V_bias']), **val)

# fname = "tmp/test_conductance_{}_with_V.hdf"
# funcs.run_simulation(lview, func, vals, dict(**params, **syst_pars), fname, 200)

In [ ]:
syst_pars = dict(a=10, angle=45, coverage_angle=180, r1=35, r2=70, shape='circle', with_shell=True)

def gap(val, syst_pars=syst_pars, params=params):
    import funcs
    val['B_x'], val['B_y'], val['B_z'] = val.pop('B')
    params = funcs.parse_params(dict(**params, **val))
    lead = funcs.make_lead(**syst_pars)
    return dict(E_gap=funcs.find_gap(lead, params), **val)

fname = "tmp/gaps_{}.hdf"
funcs.run_simulation(lview, gap, vals, dict(**params, **syst_pars), fname, 200)